## I. Import libraries

In [1]:
import json
import pandas as pd
import re
import numpy as np

### 1. Xu ly homedy dataset

In [2]:
with open (r"C:\Users\THIS PC\Desktop\apartment-price-prediction\data\raw\homedy\homedy.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
homedy_dataset = pd.DataFrame(data)

In [4]:
homedy_dataset.head()

,title,real_estate,address,price,acreage,unit_price,description,Loại hình,Hướng nhà,url,source,Tình trạng pháp lý,Số phòng ngủ,Hướng ban công,Nội thất,Căn góc,Số tầng,Đã bao gồm thuế?,Chính chủ hay không?,Đường vào
0,\nBán gấp căn 2 ngủ góc 68m2 tại phân khu Pavi...,Mua bán Căn hộ chung cư Vinhomes Ocean Park,"Xã Dương Xá, Huyện Gia Lâm, Hà Nội","3,72 Tỷ",68 m2,"54,7 tr/m2","\nCần bán gấp căn 2 ngủ góc hướng TB-ĐB, view ...",Căn hộ,Đông-Bắc,https://homedy.com/ban-can-ho-vinhomes-ocean-p...,https://homedy.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\nBán căn hộ Lạc long quân - ven Hồ Tây ( 34- ...,Bán Căn hộ Đường Trích Sài,"Bán Căn hộ Đường Trích Sài, Phường Thụy Khuê, ...","1,28 Tỷ",52 m2,"24,6 tr/m2",\nBán căn hộ chung cư mini Lạc long quân - Tây...,Căn hộ,NaN,https://homedy.com/ban-can-ho-quan-tay-ho-ha-n...,https://homedy.com,Sổ hồng riêng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,\nBán căn góc 3PN Vinhomes Smart City gần Mỹ Đ...,Mua bán Căn hộ chung cư Vinhomes Smart City,"Đường Đại Mỗ, Phường Đại Mỗ, Quận Nam Từ Liêm,...","4,8 Tỷ",75 m2,64 tr/m2,"\nBán căn hộ 3PN 75m2 giá chỉ từ 4,8 tỷ đầy đủ...",Căn hộ,NaN,https://homedy.com/ban-can-ho-vinhomes-smart-c...,https://homedy.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,\nMở bán Chung cư cao cấp Vinhomes Cổ Loa gần ...,Mua bán Căn hộ chung cư Vinhomes Cổ Loa,"Xã Cổ Loa, Huyện Đông Anh, Hà Nội",3 - 9 Tỷ,31 - 82 m2,"96,8 tr/m2",\n👉Vinhomes Global Gate Cổ Loa là dự án thấp t...,Căn hộ,NaN,https://homedy.com/ban-can-ho-vinhomes-co-loa/...,https://homedy.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\nGấp! bán nhanh Studio 28m2 Đông Bắc sát nhà ...,Mua bán Căn hộ chung cư Vinhomes Smart City,"Đường Đại Mỗ, Phường Đại Mỗ, Quận Nam Từ Liêm,...","2,15 Tỷ",28 m2,"76,8 tr/m2",\nChính chủ tin tưởng gửi bán căn hộ Studio 28...,Căn hộ,Đông-Bắc,https://homedy.com/ban-can-ho-vinhomes-smart-c...,https://homedy.com,Sổ hồng riêng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
homedy_dataset.drop(columns=['title', 'description', 'Loại hình', 'url', 'source', 'unit_price', 'Đường vào', 'Đã bao gồm thuế?', 'Nội thất', 'Hướng ban công', 'Căn góc'], inplace=True)

In [6]:
def extract_name(text):
    # Eliminate prefix "Mua bán", "Bán", "Căn hộ", "Căn hộ chung cư"
    prefixes = ["Mua bán Căn hộ chung cư", "Mua bán Căn hộ", "Bán Căn hộ chung cư", "Bán Căn hộ"]
    for prefix in prefixes:
        if text.startswith(prefix):
            return text.replace(prefix, "").strip()
    return text.strip()

homedy_dataset['real_estate'] = homedy_dataset['real_estate'].apply(extract_name)

In [7]:
# Hàm xử lý cột address
def process_address(df, column):
    # Loại bỏ tiền tố không cần thiết
    def clean_prefix(text):
        prefixes = ["Bán Căn hộ", "Mua bán Căn hộ chung cư", "Mua bán Căn hộ", "Bán Căn hộ chung cư"]
        for prefix in prefixes:
            if text.startswith(prefix):
                return text.replace(prefix, "").strip()
        return text.strip()
    
    # Áp dụng hàm loại bỏ tiền tố
    df[column] = df[column].apply(clean_prefix)
    
    # Tạo các cột mới
    df['Street'] = None
    df['Precinct'] = None
    df['District'] = None
    df['City'] = None
    
    # Phân loại từng phần của địa chỉ
    def classify_address(text):
        parts = [part.strip() for part in text.split(",")]
        street, precinct, district, city = None, None, None, None
        
        for part in parts:
            if part.startswith("Đường") or part.startswith("Phố"):
                street = part
            elif part.startswith("Phường") or part.startswith("Xã"):
                precinct = part
            elif part.startswith("Thị xã") or part.startswith("Quận") or part.startswith("Huyện"):
                district = part
            elif part == "Hà Nội":
                city = part
        
        return street, precinct, district, city
    
    # Áp dụng hàm phân loại địa chỉ vào từng hàng
    df[['Street', 'Precinct', 'District', 'City']] = df[column].apply(lambda x: pd.Series(classify_address(x)))
    
    return df

# Xử lý cột address
homedy_dataset = process_address(homedy_dataset, "address")
homedy_dataset.drop(columns=['address'], inplace=True)

In [8]:
# Hàm xử lý cột acreage
def process_acreage(df, column):
    # Tạo cột mới acreage_value và acreage_unit
    df['acreage_value'] = None
    df['acreage_unit'] = None

    # Hàm tách giá trị và đơn vị
    def extract_acreage(text):
        text = text.replace(",", ".")
        # Kiểm tra nếu chuỗi có định dạng x - y m2
        match_range = re.match(r"(\d+(\.\d+)?)\s*-\s*\d+(\.\d+)?\s*m2", text)
        if match_range:
            value = float(match_range.group(1))  # Chỉ lấy số x
            return round(value, 2), "m2"
        
        # Kiểm tra nếu chuỗi chỉ có một số và m2
        match_single = re.match(r"(\d+(\.\d+)?)\s*m2", text)
        if match_single:
            value = float(match_single.group(1))
            return round(value, 2), "m2"
        
        # Trường hợp không khớp, trả về null
        return np.nan, "m2"

    # Áp dụng hàm extract_acreage cho từng giá trị trong cột
    df[['acreage_value', 'acreage_unit']] = df[column].apply(lambda x: pd.Series(extract_acreage(x)))
    
    return df

# Xử lý cột acreage
homedy_dataset = process_acreage(homedy_dataset, "acreage")
homedy_dataset.drop(columns=['acreage'], inplace=True)

In [9]:
# Hàm xử lý cột price
def process_price(df, column):
    # Tạo cột mới price_value và price_unit
    df['price_value'] = None
    df['price_unit'] = None

    # Hàm tách giá trị và đơn vị
    def extract_price(text):
        text = text.replace(",", ".")
        text = text.lower()
        # Kiểm tra nếu chuỗi có định dạng x - y Tỷ
        match_range = re.match(r"(\d+(\.\d+)?)\s*-\s*\d+(\.\d+)?\s*tỷ", text)
        if match_range:
            value = float(match_range.group(1))  # Chỉ lấy số x
            return round(value, 2), "tỷ"
        
        # Kiểm tra nếu chuỗi chỉ có một số và Tỷ
        match_single = re.match(r"(\d+(\.\d+)?)\s*tỷ", text)
        if match_single:
            value = float(match_single.group(1))
            return round(value, 2), "tỷ"
        
        # Kiểm tra nếu chuỗi chỉ có một số và Triệu
        match_single = re.match(r"(\d+(\.\d+)?)\s*triệu", text)
        if match_single:
            value = float(match_single.group(1))
            return round(value, 2), "triệu"
        
        # Kiểm tra nếu chuỗi có định dạng x - y Triệu
        match_range = re.match(r"(\d+(\.\d+)?)\s*-\s*\d+(\.\d+)?\s*triệu", text)
        if match_range:
            value = float(match_range.group(1))  # Chỉ lấy số x
            return round(value, 2), "triệu"
        
        # Trường hợp không khớp, trả về null
        return np.nan, "tỷ"

    # Áp dụng hàm extract_acreage cho từng giá trị trong cột
    df[['price_value', 'price_unit']] = df[column].apply(lambda x: pd.Series(extract_price(x)))
    
    return df

# Xử lý cột price
homedy_dataset = process_price(homedy_dataset, "price")
homedy_dataset.drop(columns=['price'], inplace=True)

In [10]:
homedy_dataset.head()

,real_estate,Hướng nhà,Tình trạng pháp lý,Số phòng ngủ,Số tầng,Chính chủ hay không?,Street,Precinct,District,City,acreage_value,acreage_unit,price_value,price_unit
0,Vinhomes Ocean Park,Đông-Bắc,NaN,NaN,NaN,NaN,None,Xã Dương Xá,Huyện Gia Lâm,Hà Nội,68.0,m2,3.72,tỷ
1,Đường Trích Sài,NaN,Sổ hồng riêng,NaN,NaN,NaN,Đường Trích Sài,Phường Thụy Khuê,Quận Tây Hồ,Hà Nội,52.0,m2,1.28,tỷ
2,Vinhomes Smart City,NaN,NaN,NaN,NaN,NaN,Đường Đại Mỗ,Phường Đại Mỗ,Quận Nam Từ Liêm,Hà Nội,75.0,m2,4.80,tỷ
3,Vinhomes Cổ Loa,NaN,NaN,NaN,NaN,NaN,None,Xã Cổ Loa,Huyện Đông Anh,Hà Nội,31.0,m2,3.00,tỷ
4,Vinhomes Smart City,Đông-Bắc,Sổ hồng riêng,NaN,NaN,NaN,Đường Đại Mỗ,Phường Đại Mỗ,Quận Nam Từ Liêm,Hà Nội,28.0,m2,2.15,tỷ


### 2. Xu ly alonhadat.com data

In [11]:
with open (r"C:\Users\THIS PC\Desktop\apartment-price-prediction\data\raw\alonhadat\alonhadat.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

alonhadat_dataset = pd.DataFrame(data)

In [12]:
alonhadat_dataset.head()

,title,price,area,address,huong,phongAn,dgTrcNha,nhaBep,type,phapLy,sanThg,chNgang,soLau,carSlot,chDai,noBed,chinhChu
0,Bán căn hộ chung cư Phạm Văn Đồng_TP giao lưu_...,"3,9 tỷ",75 m2,"Đường Phạm Văn Đồng, Phường Cổ Nhuế 1, Quận Bắ...",_,---,---,---,Căn hộ chung cư,---,---,---,6,---,---,2,---
1,(HIẾM) Bán Nhà Tập Thể Ban Nội Chính Trung Ươn...,"3,7 tỷ",95 m2,"Phố Linh Lang, Phường Cống Vị, Quận Ba Đình, H...",_,---,5m,---,Căn hộ chung cư,---,---,---,3,---,---,3,---
2,CHÍNH CHỦ BÁN NHÀ TẬP THỂ QUỲNH MAI- TRUNG TÂM...,"3,39 tỷ",36 m2,"Đường Quỳnh Mai, Phường Quỳnh Lôi, Quận Hai B...",_,,6m,,Căn hộ chung cư,Sổ hồng/ Sổ đỏ,---,4m,3,---,9m,2,---
3,"Bán căn hộ chung cư Đại Mỗ 53m², 2PN, full nội...","2,75 tỷ",53 m2,"Đường Sa Đôi, Phường Đại Mỗ, Quận Nam Từ Liêm,...",_,---,8m,---,Căn hộ chung cư,Sổ hồng/ Sổ đỏ,---,6m,14,---,9m,2,---
4,Độc quyền quỹ căn 2-3-4PN cần bán gấp cắt lỗ t...,"5,9 tỷ",110 m2,"Đường Hồ Tùng Mậu, Phường Phú Diễn, Quận Bắc T...",_,---,---,---,Căn hộ chung cư,---,---,---,---,---,---,3,---


In [13]:
alonhadat_dataset.drop(columns=['title', 'chDai', 'chNgang', 'sanThg', 'carSlot', 'nhaBep', 'dgTrcNha', 'phongAn', 'type'], inplace=True)

In [14]:
alonhadat_dataset = process_address(alonhadat_dataset, "address")
alonhadat_dataset.drop(columns=['address'], inplace=True)

In [15]:
alonhadat_dataset = process_acreage(alonhadat_dataset, "area")
alonhadat_dataset.drop(columns=['area'], inplace=True)

In [16]:
alonhadat_dataset = process_price(alonhadat_dataset, "price")
alonhadat_dataset.drop(columns=['price'], inplace=True)

In [17]:
def process_special_nulls(df, column):
    df[column] = df[column].apply(lambda x: np.nan if x == "---" or x == "_" or x == "" or x == "N/A" else x)
    return df

alonhadat_dataset = process_special_nulls(alonhadat_dataset, "huong")
alonhadat_dataset = process_special_nulls(alonhadat_dataset, "phapLy")
alonhadat_dataset = process_special_nulls(alonhadat_dataset, "soLau")
alonhadat_dataset = process_special_nulls(alonhadat_dataset, "noBed")
alonhadat_dataset = process_special_nulls(alonhadat_dataset, "chinhChu")

In [18]:
alonhadat_dataset.head()

,huong,phapLy,soLau,noBed,chinhChu,Street,Precinct,District,City,acreage_value,acreage_unit,price_value,price_unit
0,NaN,NaN,6,2,NaN,Đường Phạm Văn Đồng,Phường Cổ Nhuế 1,Quận Bắc Từ Liêm,Hà Nội,75.0,m2,3.90,tỷ
1,NaN,NaN,3,3,NaN,Phố Linh Lang,Phường Cống Vị,Quận Ba Đình,Hà Nội,95.0,m2,3.70,tỷ
2,NaN,Sổ hồng/ Sổ đỏ,3,2,NaN,Đường Quỳnh Mai,Phường Quỳnh Lôi,Quận Hai Bà Trưng,Hà Nội,36.0,m2,3.39,tỷ
3,NaN,Sổ hồng/ Sổ đỏ,14,2,NaN,Đường Sa Đôi,Phường Đại Mỗ,Quận Nam Từ Liêm,Hà Nội,53.0,m2,2.75,tỷ
4,NaN,NaN,NaN,3,NaN,Đường Hồ Tùng Mậu,Phường Phú Diễn,Quận Bắc Từ Liêm,Hà Nội,110.0,m2,5.90,tỷ


### 3. Xu ly bds68 data

In [19]:
with open (r"C:\Users\THIS PC\Desktop\apartment-price-prediction\data\raw\bds68\bds68.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

bds68_dataset = pd.DataFrame(data)

In [20]:
bds68_dataset.head()

,Url,Source,Tên,Loại tin rao,Dự án,Vị trí,Giá,Diện tích,Mô tả,Số Phòng Ngủ,Số Phòng Tắm,Diện Tích Sử Dụng,Năm xây dựng,Đường Trước Nhà,Mặt Tiền,Số Tầng,Hướng Nhà
0,https://bds68.com.vn/ban-chung-cu/ha-noi/ha-do...,https://bds68.com.vn/,Bán 100m2 liền kề b1.4 b2.3 a2.4 chính chủ giá...,Nhà Chung Cư,Kiến Hưng,Phường Kiến Hưng Quận Hà Đông Hà Nội,7 tỷ,100 m2,Bán 100m2 liền kề b1.4 b2.3 a2.4 chính chủ giá...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://bds68.com.vn/ban-chung-cu/ha-noi/thanh...,https://bds68.com.vn/,Chính chủ bán căn 3pn dt 107m2 chung cư times ...,Nhà Chung Cư,Times Tower - HACC1 Complex Building,"Times Tower - HACC1 Complex Building, 35, Đườn...",7.6 tỷ,107 m2,Chính chủ bán căn hộ 3PN tòa chung cư Times To...,3,2,NaN,NaN,NaN,NaN,NaN,NaN
2,https://bds68.com.vn/ban-chung-cu/ha-noi/tay-h...,https://bds68.com.vn/,Bán căn hộ chung cư pentstudio west lake hanoi...,Nhà Chung Cư,PentStudio,"PentStudio, 699, Đường Lạc Long Quân, Phường P...",19 tỷ,167 m2,Bán căn hộ chung cư PentStudio West Lake Hanoi...,2,2,NaN,NaN,NaN,NaN,NaN,NaN
3,https://bds68.com.vn/ban-chung-cu/ha-noi/hoang...,https://bds68.com.vn/,Ngoại giao gửi bán căn hộ chung cư cao cấp han...,Nhà Chung Cư,Hà Nội Melody Residences,"Dự án Hà Nội Melody Residences, Đường Bằng A, ...",2.81 tỷ,74 m2,Hiện tại đầu ngoại giao Chung cư Hanoi Melody ...,2,2,NaN,NaN,NaN,NaN,NaN,NaN
4,https://bds68.com.vn/ban-chung-cu/ha-noi/bac-t...,https://bds68.com.vn/,"ĐẠI CÁT BẮC TỪ LIÊM NHÀ XÂY CHẮC CHĂN ,KHUNG C...",Nhà Chung Cư,,Đường Đại Cát Phường Thượng Cát Quận Bắc Từ Li...,6.5 tỷ,38 m2,"ĐẠI CÁT BẮC TỪ LIÊM NHÀ XÂY CHẮC CHĂN ,KHUNG C...",NaN,NaN,38 m2,2024,6m,4.5m,5,Nam


In [21]:
bds68_dataset.drop(columns=['Url', 'Source', 'Tên', 'Loại tin rao', 'Mô tả', 'Đường Trước Nhà', 'Mặt Tiền', 'Năm xây dựng', 'Diện Tích Sử Dụng'], inplace=True)

In [22]:
districts_hanoi = [
    "Ba Đình", "Hoàn Kiếm", "Tây Hồ", "Long Biên", "Cầu Giấy", 
    "Đống Đa", "Hai Bà Trưng", "Hoàng Mai", "Thanh Xuân", "Nam Từ Liêm", 
    "Bắc Từ Liêm", "Hà Đông"
]

# Hàm phân loại địa chỉ có dấu phẩy
def classify_address_with_comma(text):
    parts = [part.strip() for part in text.split(",")]
    street, precinct, district, city = None, None, None, None

    for i, part in enumerate(parts):
        if part.startswith("Đường") or part.startswith("Phố"):
            street = part
        elif part.startswith("Phường") or part.startswith("Xã"):
            precinct = part
        elif part == "Hà Nội":
            city = part
        elif part.startswith("Quận") or part.startswith("Huyện") or part.startswith("Thị xã"):
            district = part

    # Xử lý trường hợp không có từ khóa "Quận", "Huyện", "Thị xã"
    if not district:
        if precinct:
            # Lấy phần tử ngay sau precinct làm district nếu tồn tại
            precinct_index = parts.index(precinct)
            if precinct_index + 1 < len(parts) and parts[precinct_index + 1] != "Hà Nội":
                district = parts[precinct_index + 1]
        elif city == "Hà Nội":
            # Nếu có city là "Hà Nội", lấy phần trước nó làm district
            city_index = parts.index("Hà Nội")
            if city_index - 1 >= 0:
                district = parts[city_index - 1]
    # street = normalize_spaces(street)
    # precinct = normalize_spaces(precinct)

    if district in districts_hanoi:
        district = "Quận " + district
    elif district is not None and "Quận" not in district and "Thị xã" not in district and "Huyện" not in district:
        district = "Huyện " + district

    return street, precinct, district, "Hà Nội"

# Hàm phân loại địa chỉ không có dấu phẩy
def classify_address_without_comma(text):
    # Từ điển lưu các phần của địa chỉ
    address_parts = {"Street": None, "Precinct": None, "District": None, "City": None}
    
    # Các từ khóa nhận diện đơn vị và regex cải tiến
    patterns = {
        "Street": r"(Đường|Quốc lộ|Phố)\s[\w\s]+?(?=\s(Phường|Quận|Huyện|Thị xã|Thành phố|$))",
        "Precinct": r"Phường\s[\w\s]+?(?=\s(Quận|Huyện|Thị xã|Thành phố|$))",
        "District": r"(Quận|Huyện|Thị xã)\s[\w\s]+?(?=\s(Thành phố|Hà Nội|$))",
        "City": r"(Hà Nội|Thành phố\s[\w\s]+)"
    }
    
    # Tìm và lưu các phần của địa chỉ theo từ khóa
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            address_parts[key] = match.group().strip()
    
    return address_parts["Street"], address_parts["Precinct"], address_parts["District"], "Hà Nội"

# Hàm chính xử lý cột địa chỉ
def process_bds68_address(df, column):
    # Tạo các cột mới cho địa chỉ
    df['Street'] = None
    df['Precinct'] = None
    df['District'] = None
    df['City'] = None

    # Xác định và phân loại địa chỉ dựa trên định dạng của chúng
    def classify_address(text):
        if "," in text:
            # Địa chỉ có dấu phẩy
            return classify_address_with_comma(text)
        else:
            # Địa chỉ không có dấu phẩy
            return classify_address_without_comma(text)

    # Áp dụng hàm classify_address cho từng hàng
    df[['Street', 'Precinct', 'District', 'City']] = df[column].apply(classify_address).apply(pd.Series)
    
    return df

bds68_dataset = process_bds68_address(bds68_dataset, "Vị trí")
bds68_dataset.drop(columns=['Vị trí'], inplace = True)

for idx, d in bds68_dataset.iterrows():
    if d['District'] == 'Huyện Thị Trấn thường Tín' or d['District'] == 'Huyện Thường Tín':
        d['District'] = 'Thị Trấn Thường Tín'
    if d["District"] == 'Huyện Bắc aTừ Liêm':
        d['District'] = 'Quận Bắc Từ Liêm'
    if d['District'] == 'Huyện Cái Răng' or d['District']=='Huyện ' or d['District'] == "Huyện Đường Linh Đường":
        d['District'] = None
    if d['District'] == 'Huyện Cầu giấy':
        d['District'] = 'Quận Cầu Giấy'
    if d['District'] == 'Huyện Hoài   Đức':
        d['District'] = 'Huyện Hoài Đức'
    

In [23]:
bds68_dataset = process_acreage(bds68_dataset, "Diện tích")
bds68_dataset.drop(columns=['Diện tích'], inplace=True)
bds68_dataset = process_price(bds68_dataset, "Giá")
bds68_dataset.drop(columns=['Giá'], inplace=True)

In [24]:
bds68_dataset = process_special_nulls(bds68_dataset, "Dự án")

In [25]:
bds68_dataset.head()

,Dự án,Số Phòng Ngủ,Số Phòng Tắm,Số Tầng,Hướng Nhà,Street,Precinct,District,City,acreage_value,acreage_unit,price_value,price_unit
0,Kiến Hưng,NaN,NaN,NaN,NaN,None,Phường Kiến Hưng,Quận Hà Đông,Hà Nội,100.0,m2,7.00,tỷ
1,Times Tower - HACC1 Complex Building,3,2,NaN,NaN,Đường Lê Văn Lương,Phường Nhân Chính,Quận Thanh Xuân,Hà Nội,107.0,m2,7.60,tỷ
2,PentStudio,2,2,NaN,NaN,Đường Lạc Long Quân,Phường Phú Thượng,Quận Tây Hồ,Hà Nội,167.0,m2,19.00,tỷ
3,Hà Nội Melody Residences,2,2,NaN,NaN,Đường Bằng A,Phường Hoàng Liệt,Quận Hoàng Mai,Hà Nội,74.0,m2,2.81,tỷ
4,NaN,NaN,NaN,5,Nam,Đường Đại Cát,Phường Thượng Cát,Quận Bắc Từ Liêm,Hà Nội,38.0,m2,6.50,tỷ


### 4. Combine all dataframe

In [26]:
homedy_dataset.columns

Index(['real_estate', 'Hướng nhà', 'Tình trạng pháp lý', 'Số phòng ngủ',
       'Số tầng', 'Chính chủ hay không?', 'Street', 'Precinct', 'District',
       'City', 'acreage_value', 'acreage_unit', 'price_value', 'price_unit'],
      dtype='object')

In [27]:
alonhadat_dataset.columns

Index(['huong', 'phapLy', 'soLau', 'noBed', 'chinhChu', 'Street', 'Precinct',
       'District', 'City', 'acreage_value', 'acreage_unit', 'price_value',
       'price_unit'],
      dtype='object')

In [28]:
bds68_dataset.columns

Index(['Dự án', 'Số Phòng Ngủ', 'Số Phòng Tắm', 'Số Tầng', 'Hướng Nhà',
       'Street', 'Precinct', 'District', 'City', 'acreage_value',
       'acreage_unit', 'price_value', 'price_unit'],
      dtype='object')

In [29]:
column_mapping = {
    'Hướng nhà': 'huong', 'Hướng Nhà': 'huong',
    'Số phòng ngủ': 'noBed', 'Số Phòng Ngủ': 'noBed',
    'Số tầng': 'soLau', 'Số Tầng': 'soLau',
    'Căn góc': 'canGoc', 'Chính chủ hay không?': 'chinhChu',
    'Tình trạng pháp lý': 'phapLy', 'Dự án': 'duAn', 'real_estate':'duAn', 'Số Phòng Tắm': 'noBathroom'
}

In [30]:
homedy_columns = ['real_estate', 'Hướng nhà', 'Tình trạng pháp lý', 'Số phòng ngủ',
                  'Căn góc', 'Số tầng', 'Chính chủ hay không?', 'Street', 'Precinct',
                  'District', 'City', 'acreage_value', 'acreage_unit', 'price_value',
                  'price_unit']

alonhadat_columns = ['huong', 'phapLy', 'soLau', 'noBed', 'chinhChu', 'Street', 'Precinct',
                     'District', 'City', 'acreage_value', 'acreage_unit', 'price_value',
                     'price_unit']

bds68_columns = ['Dự án', 'Số Phòng Ngủ', 'Số Phòng Tắm', 'Số Tầng', 'Hướng Nhà',
                 'Street', 'Precinct', 'District', 'City', 'acreage_value', 'acreage_unit',
                 'price_value', 'price_unit']

acreage_columns = ['acreage_value', 'acreage_unit']
price_columns = ['price_value', 'price_unit']

# Dựng lại tên cột trong từng DataFrame để đồng nhất
def rename_columns(df, column_mapping):
    return df.rename(columns=column_mapping)

# Áp dụng việc đổi tên cột
homedy_df = rename_columns(homedy_dataset, column_mapping)
alonhadat_df = rename_columns(alonhadat_dataset, column_mapping)
bds68_df = rename_columns(bds68_dataset, column_mapping)

# Bây giờ ghép tất cả các DataFrame lại với nhau
apartment_dataset = pd.concat([homedy_df, alonhadat_df, bds68_df], ignore_index=True)

In [31]:
apartment_dataset = apartment_dataset.replace({None: np.nan})
apartment_dataset.drop(columns=['chinhChu'], inplace=True)

In [32]:
def process_huongNha(df, column):
    df[column] = df[column].str.replace('-', ' ')
    df[column] = df[column].replace(r'\r\n.*\xa0.*', np.nan, regex=True)
    return df

apartment_dataset = process_huongNha(apartment_dataset, 'huong')

In [33]:
apartment_dataset.head()

,duAn,huong,phapLy,noBed,soLau,Street,Precinct,District,City,acreage_value,acreage_unit,price_value,price_unit,noBathroom
0,Vinhomes Ocean Park,Đông Bắc,NaN,NaN,NaN,NaN,Xã Dương Xá,Huyện Gia Lâm,Hà Nội,68.0,m2,3.72,tỷ,NaN
1,Đường Trích Sài,NaN,Sổ hồng riêng,NaN,NaN,Đường Trích Sài,Phường Thụy Khuê,Quận Tây Hồ,Hà Nội,52.0,m2,1.28,tỷ,NaN
2,Vinhomes Smart City,NaN,NaN,NaN,NaN,Đường Đại Mỗ,Phường Đại Mỗ,Quận Nam Từ Liêm,Hà Nội,75.0,m2,4.80,tỷ,NaN
3,Vinhomes Cổ Loa,NaN,NaN,NaN,NaN,NaN,Xã Cổ Loa,Huyện Đông Anh,Hà Nội,31.0,m2,3.00,tỷ,NaN
4,Vinhomes Smart City,Đông Bắc,Sổ hồng riêng,NaN,NaN,Đường Đại Mỗ,Phường Đại Mỗ,Quận Nam Từ Liêm,Hà Nội,28.0,m2,2.15,tỷ,NaN


In [34]:
mapping = {
    'Sổ hồng riêng': 'Giấy chứng nhận quyền sở hữu đất',
    'Hợp đồng mua bán': 'Giấy tờ hợp lệ',
    'Sổ hồng': 'Giấy chứng nhận quyền sở hữu đất',
    'Sổ đỏ': 'Giấy chứng nhận quyền sở hữu đất',
    '--': np.nan,
    'Sổ hồng chung': 'Giấy chứng nhận quyền sở hữu đất',
    'Sổ trắng': 'Giấy tờ hợp lệ',
    'Giấy tờ khác': 'Giấy tờ hợp lệ',
    'Giấy chứng nhận quyền sở hữu': 'Giấy chứng nhận quyền sở hữu đất',
    'Công chứng vi bằng': 'Giấy tờ hợp lệ',
    'Sổ hồng/ Sổ đỏ': 'Giấy chứng nhận quyền sở hữu đất',
    'Giấy tờ hợp lệ': 'Giấy tờ hợp lệ',
    'Giấy phép XD': 'Giấy tờ hợp lệ'
}

apartment_dataset['phapLy'] = apartment_dataset['phapLy'].replace

In [ ]:
apartment_dataset.to_csv(r'C:\Users\THIS PC\Desktop\apartment-price-prediction\data\normalized\apartment_dataset.csv', index=False)